# Exploratory Data Analysis of the DengAI dataset

## Sections
1. [Data Ingestion](#Data_Ingestion)
2. [Summary Statistics](#Summary_Statistics)
3. Data Cleaning and Preprocessing
4. [Visualization](#Visualization)
5. Performing different tests {Correlation analysis, Dickey Fuller test, ADF, etc.}
6. [Conclusions](#Conclusions)

7. Experimenting with different models

## Importing required libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import seaborn as sns
sns.set()

from statsmodels.tsa.seasonal import seasonal_decompose as decompose
from pandas import Series
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import datetime as dt
from matplotlib.pyplot import figure
from statsmodels.tsa.seasonal import seasonal_decompose

***
<a id='Data_Ingestion'></a>
# 1. Data Ingestion

The study consists of data for two cities: __*San Juan*__ and __*Iquitos*__

The dataset consists of two parts
* __dengue_features_train.csv__ : Contains readings for all the factors considered in the study (like humidity, temperature, etc.)
* __dengue_labels_train.csv__ : Contains the number of Dengue cases reported on a weekly basis.

In [ ]:
env = 'VSCode'#'Colab'
path = '.'

if env == 'Colab':
    from google.colab import drive
    drive.mount('/content/drive')
    !ls drive/MyDrive/Dataset
    path = 'drive/MyDrive'

features = pd.read_csv(f'{path}/Dataset/dengue_features_train.csv')
features.week_start_date = pd.to_datetime(features.week_start_date) # Converting the date into a date time object

cases = pd.read_csv(f'{path}/Dataset/dengue_labels_train.csv')
testDf = pd.read_csv(f'{path}/Dataset/dengue_features_test.csv')

# Performing an Inner Join on the dataset on the common columns
df = pd.merge(features, cases, on = ["year", "weekofyear", "city"])
df.index = df.week_start_date
df.head()

## Counting number of rows and columns in the dataset

In [ ]:
print(f"Number of attributes: {len(df.columns)}")
print(f"Number of rows {len(df)}")

## As our dataset consists of two different cities, we will perform analysis on each city separately.

In [ ]:
sj_df = df[df["city"] == 'sj']
sj_df.head()

In [ ]:
iq_df = df[df["city"] == 'iq']
iq_df.head()

In [ ]:
datasets = {"San Juan" : sj_df, "Iquitos" : iq_df}
for city in datasets:
    print(f"Shape of {city}'s data = {datasets[city].shape}")

***
<a id='Summary_Statistics'></a>
# 2. Summary Statistics

In [ ]:
sj_df.describe()

In [ ]:
iq_df.describe()

## 2.1 Missing Data

In [ ]:
for city in datasets:
    print(city)
    nullVals = datasets[city].isnull()
    nullValCounts = nullVals.sum()
    print(dict(nullValCounts))
    print(f"\nFraction of data missing per column:\n{dict(nullValCounts / len(datasets[city]))}\n")
    print(f"Total Missing data = {nullValCounts.sum()}")
    print(f"Total Duplicated data = {datasets[city].duplicated().sum()}")
    print(f"Total Incomplete data = {nullVals.any(axis = 1).sum()}")
    print("------------------------------------------\n")

## 2.2 Outliers
Box-Plots to visualize outliers in the number of cases for both the cities

In [ ]:
# Box plot for every year
plt.rcParams["figure.figsize"] = 24, 15
fig, axs = plt.subplots(2, 1)

# Number of cases every year
sj_df.boxplot(by = 'year', column = ['total_cases'], ax = axs[0])
axs[0].set_title("Yearly cases in San Juan", size = 24)

iq_df.boxplot(by = "year", column = ["total_cases"], ax = axs[1])
axs[1].set_title("Yearly cases in Iquitos", size = 24)

plt.show()

In [ ]:
sj_df.insert(loc = len(sj_df.columns), column = 'month', value = sj_df['week_start_date'].dt.strftime("%b"))
year_monthly = sj_df.groupby(['year','month']).sum('total_cases')

In [ ]:
year_monthly.columns

In [ ]:
year_monthly

In [ ]:
# fig, ax = plt.subplots(figsize=(8,6))
# year_monthly['total_cases'].plot(kind='kde',ax=ax)

year_monthly.unstack('month')['total_cases'].plot(kind = "bar")
plt.show()

In [ ]:
print(year_monthly['total_cases'])

***
<a id='Visualization'></a>
# 4. Visualization

## 4.1 Line Plots

### 4.1.1 Line plot to vizualize `Number of cases vs time`, `Humidity vs time` and `temperature vs time`

In [ ]:
all_concerns = [
  'total_cases',
  'reanalysis_specific_humidity_g_per_kg',
  'reanalysis_avg_temp_k'
]

labels = [
  'Dengue Cases',
  'Humidity',
  'Average Temperature'
]

colors = ["red", "green"]

plt.rcParams['figure.figsize'] = (27, 27) # rc -> runtime config
fig,ax = plt.subplots(len(all_concerns) * len(datasets), 1) 

for index, city in enumerate(datasets):
  for i,concern in enumerate(all_concerns):
    ax[i + index*len(all_concerns)].plot(
        datasets[city].index,
        datasets[city][concern],
        color = colors[0 + index],
        label = f"{labels[i]} in {city}"
    )
    ax[i + index*len(all_concerns)].set_title(f"{labels[i]} vs time")
    ax[i + index*len(all_concerns)].legend()

plt.show()

### 4.1.2 Finding the number of yearly cases in both cities

In [ ]:
sj_df_yearly = pd.DataFrame(
    {
        "year" : np.unique(sj_df['year']),
        "total_cases" : sj_df.groupby(["year"])["total_cases"].sum()
    }
)

iq_df_yearly = pd.DataFrame(
    {
        "year" : np.unique(iq_df['year']),
        "total_cases" : iq_df.groupby(["year"])["total_cases"].sum()
    }
)

plt.rcParams['figure.figsize'] = (25, 7) #rc -> runtime config
fig, axs = plt.subplots(1, 2) 

axs[0].plot(sj_df_yearly.year, sj_df_yearly.total_cases)
axs[0].set_title("Yearly Cases in San Juan")

axs[1].plot(iq_df_yearly.year, iq_df_yearly.total_cases)
axs[1].set_title("Yearly Cases in Iquitos")

plt.show()

## 4.2 Histograms
Understanding the distribution of data for `total number of cases, specific humidity and air temperature in both cities`

**TODO FOR ALL IMPORTANT FEATURES**

In [ ]:
all_concerns = [
  'total_cases',
  'reanalysis_specific_humidity_g_per_kg',
  'reanalysis_avg_temp_k'
]

labels = [
  'Dengue Cases',
  'Humidity',
  'Average Temperature'
]

plt.rcParams["figure.figsize"] = 30, 20
fig, axs = plt.subplots(len(all_concerns), len(datasets)) 
plt.subplots_adjust(hspace = 0.8) #space between plots

for index, city in enumerate(datasets):
    for i,concern in enumerate(all_concerns):
        sns.histplot(datasets[city][concern], ax = axs[i, index], kde = True)
        axs[i, index].set_title(f"{labels[i]} in {city}",size=14)
        

plt.show()

The Dengue cases in San Juan and Iquitos are both right skewed.

Humidity is almost right left skewed while the average temperature are subbotin distribution due to the plateu in between

## 4.3 Correlation matrices

In [ ]:
f, ax = plt.subplots(figsize=(10, 10))
corr_cases = sj_df.corr()
sns.heatmap(corr_cases, square = True, cmap = 'coolwarm', ax = ax)
plt.title('Correlation Matrix for San Juan', fontsize = 16)
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(10, 10))
corr_cases = iq_df.corr()
sns.heatmap(corr_cases, square = True, cmap = 'coolwarm', ax = ax)
plt.title('Correlation Matrix for Iquitos', fontsize = 16)
plt.rcParams['figure.figsize'] = (5, 5) #rc -> runtime config
plt.show()

***
<a id='Conclusions'></a>
# 6. Conclusions

- How many rows and attributes?
    - San Juan: `(936, 25)`.
    - Iquitos : `(520, 25)`.
- How many missing data and outliers?
    - San Juan: `380 Missing`.
    - Iquitos : `168 Missing`.
- Any inconsistent, incomplete, duplicate or incorrect data?
    - All records in both the datasets are unique.
    - Number of incomplete rows in San Juan's data: `209`.
    - Number of incomplete rows in Iquitos's data : `48`.
- Are the variables correlated to each other?
    - San Juan: ``.
    - Iquitos : ``.

- Are any of the preprocessing techniques needed: rolling average, continuum cubic spline curve, dimensionality reduction, range transformation, standardization, etc.?
    - San Juan: ``.
    - Iquitos : ``.
    
- Does PCA help visualize the data? Do we get any insights from histograms/bar charts/line plots, etc.?

In [ ]:
def imputeMissing(dfs):
    for adf in dfs:
        for col in adf.columns[adf.isna().any()].tolist():
            adf[col].interpolate(method="time",inplace=True) # Why inplace?
    return iq_df,sj_df

iq_df_imputed, sj_df_imputed = imputeMissing([iq_df,sj_df])

In [ ]:
sj_df.columns

In [ ]:
def pcaAnalysis(dfs_list,num_components): #work on the imputed data
    dfs,original_dfs = dfs_list

    details = {
        0:['iq','red'],
        1:['sj','blue']
    }

    sc = StandardScaler()
    pca = PCA()
    res = []
    
    #number of components to keep can be explained with the help of 
    #explained variance ratio as a function of the number of components
    #it is the percentage of variance attributed to each of the selected component
    #hence we shld go on adding components until the total variance ratio is upto 80% or so to avoid overfitting

    for df_indx,adf in enumerate(dfs):
        adf = adf.drop(['city', 'month'], axis = 1, errors = 'ignore')
        
        #convert datetime to ordinal value from datetime object for normalization
        adf['week_start_date'] = adf['week_start_date'].map(dt.datetime.toordinal)
        adf = sc.fit_transform(adf)
        adf_pca = pca.fit(adf)
        
        plt.plot(
            np.cumsum(pca.explained_variance_ratio_),
            label = details[df_indx][0],
            color = details[df_indx][1],
        )

        pca = PCA(num_components)
        adf_pca = pca.fit_transform(adf)

        #get the most important features
        #pca.components_ is a list of eigenvector magnitudes for each priciple component
        most_imp = [np.abs(pca.components_[i]).argmax() for i in range(pca.n_components_)]
        most_imp_names = [original_dfs[df_indx].columns[most_imp[i]] for i in range(pca.n_components_)]
        res.append(most_imp_names)

    plt.xlabel("number of components")
    plt.ylabel('cumulative explained variance')
    plt.legend()
    plt.plot()
    return res

important_columns = pcaAnalysis([[iq_df_imputed,sj_df_imputed],[iq_df,sj_df]],15)

In [ ]:
def getCorrelationAtWeeksLag(df,max_lag):
    target = 'total_cases'
    df = df.drop(['city','year','week_start_date','weekofyear'], axis = 1)
    lagged_correlation = pd.DataFrame.from_dict(
    {x: [df[target].corr(df[x].shift(-t)) for t in range(max_lag+1)] for x in df.columns})
    return lagged_correlation,max_lag


lagged_correlation,max_lag = getCorrelationAtWeeksLag(iq_df,28)

In [ ]:
lagged_correlation

In [ ]:
def plotCorrelationAtLags(lagged_correlation,max_lag):
    plt.rcParams["figure.figsize"] = (20,20)

    for col in lagged_correlation.columns:
        if col!='total_cases':
            y= lagged_correlation[col]
            x = range(0,max_lag+1)
            plt.plot(x,y,label=f'{col}')
            plt.legend()
            plt.xlabel("Lag weeks")
            plt.ylabel("Correlation value with total_cases")
    plt.show()

plotCorrelationAtLags(lagged_correlation,max_lag)

Seasonality Decomposition 

In [ ]:
res = seasonal_decompose(iq_df_imputed['total_cases'], model="additive", period=4)
res.plot()
plt.show()

Inference 
->Trend : slightly increasing trend
-> Has seasonality

## XGBoost

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from xgboost import XGBRegressor, plot_importance

In [ ]:
trainDfiq = iq_df_imputed
trainDfsj = sj_df_imputed

In [ ]:
def train(trainDf):
    unwantedCols = ['month', 'week_start_date', 'year', 'weekofyear', 'week_start_date', 'city', 'total_cases']#, 'Volume']

    x = trainDf.drop(unwantedCols, axis = 1, errors = 'ignore')
    y = trainDf['total_cases']

    x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size = 0.2, random_state = 42)

    # Parameters for Grid Search
    param_grid = dict(
        preprocessor = [MinMaxScaler(), RobustScaler(), StandardScaler()]
    )

    xgbModel = XGBRegressor(
        # booster = 'gblinear',
        # objective ='reg:squarederror',
        random_state = 1, 
        n_estimators = 500, 
        learning_rate = 0.1
    )

    xgbPipeline = Pipeline(
        steps = [
            ('preprocessor', RobustScaler()),
            ('model', tunedXGB)
        ]
    )

    grid = GridSearchCV(xgbPipeline, param_grid = param_grid, cv = 4, verbose = 3)
    grid.fit(x, y)
    print("\nBest Model:")
    print(grid.best_estimator_)

    xgbPipeline = grid.best_estimator_

    scores = -1 * cross_val_score(xgbPipeline, x, y, cv = 5, scoring = 'neg_mean_absolute_error', verbose = 1)
    print(f'Mean MAE: {scores.mean()} ({scores.std()})')

    print(f"Training Score  : {xgbPipeline.score(x_train, y_train)}")
    print(f"Validation Score: {xgbPipeline.score(x_valid, y_valid)}")

    print(x.columns)
    xgbPipeline.steps[1][1].get_booster().feature_names = list(x.columns)
    plot_importance(xgbPipeline.steps[1][1].get_booster())
    plt.show()

    return xgbPipeline


In [ ]:
sjModel = train(sj_df_imputed)

In [ ]:
iqModel = train(iq_df_imputed)

## Predicting on the Test Dataset

In [ ]:
testSJ = testDf[testDf.city == 'sj'].drop(unwantedCols, axis = 1, errors = 'ignore')
testIQ = testDf[testDf.city == 'iq'].drop(unwantedCols, axis = 1, errors = 'ignore')

SJPreds = testDf[testDf.city == 'sj'][['city', 'year', 'weekofyear']]
IQPreds = testDf[testDf.city == 'iq'][['city', 'year', 'weekofyear']]

def predict(df, outputFile, sjModel, iqModel):
    SJPreds['total_cases'] = sjModel.predict(testSJ).astype(int)
    IQPreds['total_cases'] = iqModel.predict(testIQ).astype(int)

    concatenated = pd.concat((SJPreds, IQPreds))
    concatenated.to_csv(f'{path}/Predictions/{outputFile}.csv', index=False)